# Programar pipelines

In [ ]:
pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Cargar los componentes

In [ ]:
from azure.ai.ml import load_component

ing_prep_data = ml_client.components.get(name="ing_prep_data_housing_ric", version=3)
train_model_housing = ml_client.components.get(name="train_model_housing_ric", version=1)
eval_model_housing = ml_client.components.get(name="eval_model_housing_ric", version=8)

## Build del pipeline

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline


@pipeline(name="house_pricing_pipeline", description="Pipeline to train a house pricing model")
def house_pricing(input_data):
    clean_data = ing_prep_data(input_data=input_data)
    train_model = train_model_housing(input_data=clean_data.outputs.output_data, alpha=0.5)
    eval_model = eval_model_housing(input_data=train_model.outputs.output_data, model_input=train_model.outputs.model_output)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
        "pipeline_job_output_data": train_model.outputs.output_data,
    }

In [ ]:
# definir job de pipeline
pipeline_job = house_pricing(Input(type=AssetTypes.URI_FILE, path="azureml:housing_prices:2"))

# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"

# set pipeline level compute
pipeline_job.settings.default_compute = "ricardoenm2"

# set pipeline level datastore
pipeline_job.settings.default_datastore = "training_data_ric"
pipeline_job.settings.force_rerun = True

In [ ]:
print(pipeline_job)

## Programar el job

In [ ]:
from azure.ai.ml.entities import JobSchedule
from azure.ai.ml.entities import RecurrenceTrigger
from datetime import datetime


schedule_name = "run_ev_five_min_housing_pipe_ric"

recurrence_trigger = RecurrenceTrigger(
    frequency="minute",
    interval=5,
    start_time=datetime.utcnow()
)

job_schedule = JobSchedule(
    name=schedule_name,
    trigger=recurrence_trigger,
    create_job=pipeline_job
)

job_schedule = ml_client.schedules.begin_create_or_update(job_schedule).result()